#### モデルフリーの強化学習
１．Q-Learning\
    Q-learningはTD学習の中でも方策オフ型の手法です。方策はε-greedyなりsoftmaxなりを使いますので、方策を改善することはありません。


##### Q-Learningの学習フロー
１．Q関数の初期化、方策の決定、初期状態の決定\
２．方策に従って報酬、次の状態の観測\
３．TD Errorの計算\
$\delta_t:=r_t+\gamma \max _{a^{\prime} \in \mathcal{A}} \hat{Q}\left(s_{t+1}, a^{\prime}\right)-\hat{Q}\left(s_t, a_t\right)$\
４．Q関数の更新\
$\hat{Q}\left(s_t, a_t\right):=\hat{Q}\left(s_t, a_t\right)+\alpha_t \delta_t$
        
##### Q学習をJaxと行列形式で解いていく
すごい人の方法をまねて実装していきましょう!()
実装の方法





２．SARSA

３．Actor Critic